In [1]:
# Variable
W_max = 19.5;
W_min = 10;
import xlrd
import xlsxwriter
import os


## excel parameter
# parameter schedule end Index
Schedule_endIndex = 289
# student schedule end Index
EndTime = 39;

# function for formating
def timeFormating(num):
    n = str(num)
    time = n.split(".")
    if time[1]=='5':
        time[1]='50' 
    min=str(int(time[1])*60/100).split(".")[0]
    if min =='0':
        min='00'
    if int(time[0]) <12:
        return time[0]+":"+min+"am"
    elif int(time[0])==12:
        return time[0]+":"+min+"pm"
    elif int(time[0])> 24:
        return str(int(time[0])-24)+":"+min+"am"
    elif int(time[0]) >12:
        return str(int(time[0])-12)+":"+min+"pm"
    
def dayFormating(num):
    if num==1:
        return "Mon"
    elif num ==2:
        return "Tue"
    elif num==3:
        return "Wed"
    elif num==4:
        return "Thu"
    elif num==5:
        return "Fri"
    elif num==6:
        return "Sat"
    elif num==7:
        return "Sun"

loc = ("schedule.xls")
wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
x = list(range(1, Schedule_endIndex))
#initial data
startTime = []
processStartTime = []
endTime = []
shiftslength = []
days = []
shiftstations = []
station = []

# get shift data
for i in x:
    timeStarted = sheet.cell_value(i,1)
    startTime.append(timeStarted)
    if str(timeStarted)[-3:] ==".25" or str(timeStarted)[-3:] ==".75" :
        processStartTime.append(timeStarted-0.25)
    else :
        processStartTime.append(timeStarted)  
    endTime.append(sheet.cell_value(i,2))
    shiftslength.append(sheet.cell_value(i,3))
    days.append(sheet.cell_value(i,4))
    shiftstations.append(sheet.cell_value(i,5))
    if sheet.cell_value(i,5) not in station:
        station.append(sheet.cell_value(i,5))

shifts = list(range(0, Schedule_endIndex-1))
# get all students availability and prefer time
studentName = []
p = []
#allAvailability[i]:availability for student i
#allAvailability[i][j]:availability for student i on day j
#allAvailability[i][j][k]:availability for student i on day j at time k
allAvailability = []
dirs = os.listdir("studentAvailability")
for file in dirs:
    if (not file.startswith("~$")) and (not file.startswith(".")) and (not file.startswith("_")):
        studentName.append(file.split(".")[0])
        # get each student availability
        wbp=xlrd.open_workbook("studentAvailability/"+file)
        sheetperson = wbp.sheet_by_index(0)
        # get prefer working time
        p.append(sheetperson.cell_value(1,8))
        time_slot = list(range(1, EndTime))
        stuAvailability = []
        for j in list(range(1,8)):
            day_slot = []
            for i in time_slot:
                day_slot.append(sheetperson.cell_value(i,j))
            stuAvailability.append(day_slot)
        allAvailability.append(stuAvailability)
# get whether shift is available for each student
a=[]
for i in range(len(studentName)):
    stuShiftAvailability = []
    stuAvailability = allAvailability[i]
    #loop all shifts
    for j in shifts:
        shiftstartTime = startTime[j]
        shiftLength = shiftslength[j]
        day = int(days[j]-1)
        startTimeSlot = int((shiftstartTime-7)/0.5)
        lengthSlot = int(shiftLength/0.5)
        # get student availability for the specific day
        stuDayAvailability = stuAvailability[day]
        # loop one single shift
        available = 1;
        for k in list(range(startTimeSlot,startTimeSlot+lengthSlot)):
            if(stuDayAvailability[k] ==0):
                available=0
                break
        stuShiftAvailability.append(available)
    a.append(stuShiftAvailability)

notavailable = []
for i in shifts:
    for j in range(i, len(shifts)):
        if days[i] == days[j] and i !=j:
            if startTime[i] <= startTime[j] and endTime[i] > startTime[j]:
                notavailable.append([i,j])
            elif endTime[j] > startTime[i] and endTime[j] <= endTime[i]:
                notavailable.append([i,j])

In [2]:
#del allAvailability
# model building
from gurobipy import *
import statistics
import math

def buildModel(ohm_1,ohm_2):
    m = Model()
    # initial variable
    #x[i][j] assign student i to shift j
    x=[]
    for i in range(len(studentName)):
        y = [None]*len(shifts)
        x.append(y)
        for j in shifts:
            x[i][j] = m.addVar(vtype=GRB.BINARY)
    m.update()

    # add constrain
    for i in range(len(studentName)):
        expr = LinExpr()
        for j in shifts:
            if a[i][j] ==0:
                m.addConstr(x[i][j]==0)
            #expr.add(x[i][j],shiftslength[j])
        expr.addTerms(shiftslength,[x[i][j] for j in shifts])
        m.addConstr(expr<=W_max)
        m.addConstr(expr>=W_min)

        for k in notavailable:
            m.addConstr(x[i][k[0]]+x[i][k[1]] <= 1)

    for j in shifts:
        expr = LinExpr()
        expr.addTerms([1]*len(studentName),[x[i][j] for i in range(len(studentName))])
        #
        #for i in range(len(studentName)):
        #    expr.add(x[i][j],1)
        m.addConstr(expr<=1)

    # goal
    # F1
    F1 = 0
    F2 = 0
    for i in range(len(studentName)):
        scheduleTime = 0
        stuStation = []
        unique = []
        lateNightshift = 0
        for j in shifts:
            if x[i][j] == 1:
                scheduleTime += x[i][j]*shiftslength[j]
                stuStation.append(shiftstations[j])
                if shiftstations[j] not in unique :
                    unique.append(shiftstations[j])

                if shiftstations[j] == "Late Night":
                    lateNightshift+=1

        F1 += (scheduleTime-p[i])*(scheduleTime-p[i])
        F2 += len(stuStation)-lateNightshift-len(unique)
    # Define the objective function
    m.Params.MIPGapAbs =2.5
    m.Params.MIPFocus=3
    m.Params.ImproveStartTime = 50
    m.Params.TimeLimit = 600
    m.Params.OutputFlag = 0
    m.setObjective((ohm_1*F1+ ohm_2*F2)/len(studentName), GRB.MINIMIZE)
    m.optimize()

    #check objective function 1
    #print("Result for objective function 1")
    difference = 0
    for j in range(0,len(studentName)):
        writeMessage = ''
        totalstuHour = 0
        for i in shifts:
            if (type(x[j][i]) != int) and (x[j][i].x==1) :
                totalstuHour += x[j][i].x*shiftslength[i]
        difference+=abs(totalstuHour-p[j])
    #print("Total difference is "+str(difference))
    #print()
    #print()
    # check objective function 2
    #print("Result for objective function 2")
    totalStation = 0
    totalunique = 0
    for i in range(len(studentName)):
        stuStation = []
        unique = []
        lateNightshift = 0
        for j in shifts:
            if x[i][j].x == 1:
                scheduleTime += x[i][j].x*shiftslength[j]
                stuStation.append(shiftstations[j])
                if shiftstations[j] not in unique :
                    unique.append(shiftstations[j])
                if shiftstations[j] =="Late Night":
                    lateNightshift+=1
        totalStation+=len(stuStation)
        totalunique+=len(unique)+lateNightshift
    stationDifference = totalStation-totalunique
    #print("Total\t "+str(totalStation)+"\t\t"+str(totalunique)+"\t\t\t"+str(totalStation-totalunique))
    #print()
    #print()
    F1 = 0
    F2 = 0
    for i in range(len(studentName)):
        scheduleTime = 0
        stuStation = []
        unique = []
        lateNightshift = 0
        for j in shifts:
            if x[i][j].x == 1:
                scheduleTime += x[i][j].x*shiftslength[j]
                stuStation.append(shiftstations[j])
                if shiftstations[j] not in unique :
                    unique.append(shiftstations[j])

                if shiftstations[j] == "Late Night":
                    lateNightshift+=1
        if scheduleTime-p[i] >= 0:
            F1 += scheduleTime-p[i]
        else:
            F1 += p[i]-scheduleTime  
        F2 += len(stuStation)-lateNightshift-len(unique)
    objF = (ohm_1*F1 + ohm_2*F2)/len(studentName)
    return [difference, stationDifference,objF]


In [3]:
import time
start = time.time()
testohmmatrix = []
error = []
ohmList = []
for i in range(1,10):
    for j in range(1,10):
        ohmList.append([i,j])
for i in ohmList:
    print()
    print("Testing case for ohm_1 =",i[0]," and ohm_2 =",i[1])
    try:
        objlist = buildModel(i[0],i[1])
        testohmmatrix.append(objlist)
    except:
        message = "Testing case for ohm_1 = "+str(i[0])+" and ohm_2 = "+str(i[1])+" not solvable."
        error.append(message)
end = time.time()
print()
print("Total running time:")
print(end - start)
        


Testing case for ohm_1 = 1  and ohm_2 = 1
Academic license - for non-commercial use only
Changed value of parameter MIPGapAbs to 2.5
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter ImproveStartTime to 50.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100

Testing case for ohm_1 = 1  and ohm_2 = 2
Changed value of parameter MIPGapAbs to 2.5
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter ImproveStartTime to 50.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100

Testing case for ohm_1 = 1  and ohm_2 = 3
Changed value of parameter MIPGapAbs to 2.5
   P


Testing case for ohm_1 = 3  and ohm_2 = 2
Changed value of parameter MIPGapAbs to 2.5
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter ImproveStartTime to 50.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100

Testing case for ohm_1 = 3  and ohm_2 = 3
Changed value of parameter MIPGapAbs to 2.5
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter ImproveStartTime to 50.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100

Testing case for ohm_1 = 3  and ohm_2 = 4
Changed value of parameter MIPGapAbs to 2.5
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-

Changed value of parameter MIPGapAbs to 2.5
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter ImproveStartTime to 50.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100

Testing case for ohm_1 = 5  and ohm_2 = 4
Changed value of parameter MIPGapAbs to 2.5
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter ImproveStartTime to 50.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100

Testing case for ohm_1 = 5  and ohm_2 = 5
Changed value of parameter MIPGapAbs to 2.5
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Changed value of parameter MIPFocus to 3

   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter ImproveStartTime to 50.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100

Testing case for ohm_1 = 7  and ohm_2 = 5
Changed value of parameter MIPGapAbs to 2.5
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter ImproveStartTime to 50.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100

Testing case for ohm_1 = 7  and ohm_2 = 6
Changed value of parameter MIPGapAbs to 2.5
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Chan

Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter ImproveStartTime to 50.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100

Testing case for ohm_1 = 9  and ohm_2 = 6
Changed value of parameter MIPGapAbs to 2.5
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter ImproveStartTime to 50.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100

Testing case for ohm_1 = 9  and ohm_2 = 7
Changed value of parameter MIPGapAbs to 2.5
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter ImproveStartTime to 50.0
   Pre

In [5]:
test_ohm_1=1
test_ohm_2=1
for i in testohmmatrix :
    if test_ohm_2==10:
        test_ohm_2=1
        test_ohm_1+=1
    print("Ohm1 =",test_ohm_1,",Ohm2 =",test_ohm_2,"F1 =",i[0],"F2 = ",i[1])
    test_ohm_2+=1

Ohm1 = 1 Ohm2 = 1 F1 = 184.25 F2 =  36
Ohm1 = 1 Ohm2 = 2 F1 = 177.75 F2 =  39
Ohm1 = 1 Ohm2 = 3 F1 = 172.75 F2 =  33
Ohm1 = 1 Ohm2 = 4 F1 = 184.25 F2 =  36
Ohm1 = 1 Ohm2 = 5 F1 = 184.25 F2 =  36
Ohm1 = 1 Ohm2 = 6 F1 = 184.25 F2 =  36
Ohm1 = 1 Ohm2 = 7 F1 = 184.25 F2 =  36
Ohm1 = 1 Ohm2 = 8 F1 = 184.25 F2 =  36
Ohm1 = 1 Ohm2 = 9 F1 = 184.25 F2 =  36
Ohm1 = 2 Ohm2 = 1 F1 = 169.25 F2 =  34
Ohm1 = 2 Ohm2 = 2 F1 = 162.25 F2 =  32
Ohm1 = 2 Ohm2 = 3 F1 = 161.75 F2 =  33
Ohm1 = 2 Ohm2 = 4 F1 = 162.75 F2 =  37
Ohm1 = 2 Ohm2 = 5 F1 = 162.75 F2 =  37
Ohm1 = 2 Ohm2 = 6 F1 = 162.75 F2 =  40
Ohm1 = 2 Ohm2 = 7 F1 = 161.75 F2 =  41
Ohm1 = 2 Ohm2 = 8 F1 = 162.75 F2 =  40
Ohm1 = 2 Ohm2 = 9 F1 = 164.25 F2 =  32
Ohm1 = 3 Ohm2 = 1 F1 = 157.25 F2 =  43
Ohm1 = 3 Ohm2 = 2 F1 = 157.25 F2 =  42
Ohm1 = 3 Ohm2 = 3 F1 = 157.25 F2 =  42
Ohm1 = 3 Ohm2 = 4 F1 = 157.25 F2 =  45
Ohm1 = 3 Ohm2 = 5 F1 = 157.25 F2 =  43
Ohm1 = 3 Ohm2 = 6 F1 = 157.25 F2 =  41
Ohm1 = 3 Ohm2 = 7 F1 = 157.25 F2 =  41
Ohm1 = 3 Ohm2 = 8 F1 = 15

In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
x=[i for i in range(1,10)]
y=[i for i in reversed(range(1,10))]
z1=[i[0] for i in testohmmatrix]
z2=[i[1] for i in testohmmatrix]
z3=[i[2] for i in testohmmatrix]
minZ1 = min(z1)
minZ2 = min(z2)
minZ3 = min(z3)
minZ1index = z1.index(minZ1)
minZ2index = z2.index(minZ2)
minZ3index = z3.index(minZ3)
fig = plt.figure()
ax = fig.gca(projection='3d')
for i, value in enumerate(x):
    if i == minZ1index:
        ax.scatter(y[i], x[i], z1[i], s=20, c='r')
    else:
        ax.scatter(y[i], x[i], z1[i], s=20, c='b')

ax.set_xlabel('Ohm_2')
ax.set_ylabel('Ohm_1')
ax.set_zlabel('F1')
plt.show()
print("Minimum F1 occur when ohm_1 =",x[minZ1index],"ohm_2 =",y[minZ1index])
fig = plt.figure()
ax = fig.gca(projection='3d')
for i, value in enumerate(x):
    if i == minZ2index:
        ax.scatter(y[i], x[i], z2[i], s=20, c='r')
    else:
        ax.scatter(y[i], x[i], z2[i], s=20, c='b')

ax.set_xlabel('Ohm_2')
ax.set_ylabel('Ohm_1')
ax.set_zlabel('F2')
plt.show()
print("Minimum F2 occur when ohm_1 =",x[minZ2index],"ohm_2 =",y[minZ2index])
fig = plt.figure()
ax = fig.gca(projection='3d')
for i, value in enumerate(x):
    if i == minZ3index:
        ax.scatter(y[i], x[i], z3[i], s=20, c='r')
    else:
        ax.scatter(y[i], x[i], z3[i], s=20, c='b')

ax.set_xlabel('Ohm_2')
ax.set_ylabel('Ohm_1')
ax.set_zlabel('F')
plt.show()
print("Minimum F occur when ohm_1 =",x[minZ3index],"ohm_2 =",y[minZ3index])